In [12]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib

In [15]:
model_english = joblib.load('train.joblib')
model_francais = joblib.load('train2.joblib')

On va d'abord extraire les titres des articles d'un site d'information français, nous avons choisi le site [Le Monde](https://www.lemonde.fr/)

In [16]:
url = 'https://www.lemonde.fr/'

In [17]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)

In [18]:
titres = None
if response.status_code == 200:
    # Utiliser Beautiful Soup pour extraire le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Exemple : Extraire tous les titres d'articles
    titres = soup.find_all('p', class_='article__title')

    # Afficher les titres
    for title in titres:
        print(title.text.strip())
        print("\n")
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")
for i in range(len(titres)): 
    titres[i] = titres[i].text.strip()

Le patron de Boeing reconnaît une « erreur » après la défaillance lors du vol Alaska Airlines


Google, Apple vs Epic Games : « Le monopole des géants du numérique s’érode trop doucement »


Cryptomonnaies : « Le bitcoin est au départ l’argent des pirates et des anarchistes du Net »


COP28 : « Comment est né ce sentiment de satisfaction et d’héroïsme qui entoure les négociateurs »


Israël-Palestine : que veut vraiment le Hamas ?


Pourquoi l’avenir du Moyen-Orient dépend autant d’Israël


En images : l’Ukraine touchée par « un nombre record de missiles » lancés par la Russie


Les coulisses de notre enquête « Narco Business » sur le trafic de fentanyl des cartels mexicains


Les risques du pari Gabriel Attal


En ces temps tragiques, que nous est-il permis d’espérer ?


« Emmanuel Macron a dû enclencher l’opération survie, moins de deux ans après sa réélection »


Mort de Thomas à Crépol : « L’utilisation politicienne d’un fait divers et la stigmatisation d’une population ne peuvent 

On va maintenant extraire les titres des articles d'un site d'information international, nous avons choisi le site américain [The Washington Post](https://www.washingtonpost.com/)

In [19]:
url2 = 'https://www.washingtonpost.com/'

In [20]:
response2 = requests.get(url2, headers=headers)

In [21]:
titles = None
if response2.status_code == 200:
    # Utiliser Beautiful Soup pour extraire le contenu HTML de la page
    soup = BeautifulSoup(response2.content, 'html.parser')

    # Exemple : Extraire tous les titres d'articles
    titles = soup.find_all('h2', class_='wpds-c-iiQaMf wpds-c-iiQaMf-ihElTZa-css')
    titles = titles + soup.find_all('h2', class_='wpds-c-iiQaMf wpds-c-iiQaMf-ibYgSwf-css')

    # Afficher les titres
    for title in titles:
        print(title.text.strip())
        print("\n")
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")
for i in range(len(titles)): 
    titles[i] = titles[i].text.strip()

Why the Iowa caucuses matter, and why they’re disparaged by critics


Alaska Airlines to cancel all Boeing 737 Max 9 flights through Saturday


Hunter Biden makes surprise appearance at House contempt hearing


How well are you aging? How to discover your ‘fitness’ age.


Should you take a magnesium supplement? Here’s what the science says.


How to use rotisserie chicken as a shortcut to easy weeknight dinners


7 ways to upgrade your home without replacing anything


1 dead, several captured in attack on U.N. helicopter in Somalia, official says


GOP governors in 15 states reject summer food money for kids


White House weighs menthol ban amid dueling health, political pressures


House GOP pivots to impeach Mayorkas as talks over border security languish


The justices will keep Trump on the ballot. Here’s how (and why) they’ll do it.


The best (and weirdest) tech we found at CES 2024


Meet the woman who transformed Sam Altman into the avatar of AI


The story behind the White Ho

Desormais, on va effectuer un analyse de sentiment sur les réponses qu'on a obtenu ci dessus. 
L'objectif est de déterminer un ratio de titre d'article positifs/négatifs (en pourcentage)

Ici, on a choisi un modèle de type SVC. Dans tous les cas, il nous fallait un modèle capable de catégoriser les data et parmi ceux testés, c'est SVC qui a fourni le meilleur résultat. On utilise un pipeline pour coupler le modèle SVC avec un TfidfVectorizer permettant de traiter les données textuelles.

In [22]:
# Utilisez le modèle pour prédire les labels
new_predictions = model_english.predict(titles)
print(new_predictions)

[1 1 1 5 1 5 5 1 1 1 1 5 5 5 1 5 1 1 1 1 1 1 1 1 5 5 1 1 1 1 5 1 1 1 1 1 5
 5 1 5 1 1 1 1 1 5 1 5 5 1 1 1 1 5 2 5 1 1 5 1 1 1 1 5 1 1 1 1 1 5 1 5 1 1
 1 5 5 1 5 1 5 1 1 5 1 1 1 1 1 5 1 1 1 1 5]


In [23]:
nb_0 = 0
nb_1 = 0
for phrase, prediction in zip(titles, new_predictions):
    if prediction==1 or prediction==2 :
        nb_0 += 1
    else :
        nb_1 += 1
print("Positive or neutral news :", round(nb_1/(nb_0+nb_1) * 100,2), "%")
print("Negative news :", round(nb_0/(nb_0+nb_1) * 100,2), "%")

Positive or neutral news : 30.53 %
Negative news : 69.47 %


In [24]:
nouvelles_predictions = model_francais.predict(titres)
print(nouvelles_predictions)

[0 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1
 1 0 0 1]


In [25]:
nb_0 = 0
nb_1 = 0
for phrase, prediction in zip(titres, nouvelles_predictions):
    if prediction==0:
        nb_0 += 1
    else :
        nb_1 += 1
print("Infos positives ou neutres:", round(nb_1/(nb_0+nb_1) * 100,2), "%")
print("Infos négatives :", round(nb_0/(nb_0+nb_1) * 100,2), "%")

Infos positives ou neutres: 73.17 %
Infos négatives : 26.83 %
